## Iniciando Modelo de Entrenamiento
Para esta parte, el articulo menciona el uso de modelo orientado a Random Forest con Regresion, actualmente ya se tiene el dataset final con mas de 34000 registros y 72 caracteristicas.
Se utilizara TensorFlow corriendo con Cuda para el entrenamiento y testeo

In [1]:
import tensorflow as tf

print(tf.config.list_physical_devices('GPU'))

2024-11-26 22:50:29.086846: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732679429.173665    1053 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732679429.197436    1053 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-26 22:50:29.405524: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
print(tf.__version__)

2.18.0


In [3]:
import torch

print(torch.cuda.is_available())

True


### Importamos el dataset

In [4]:
import pandas as pd
import numpy as np

In [5]:
dataset_df = pd.read_csv("main_dataset_without_glucose.csv")
dataset_df.head()

,datetime,nivel,gender,hba1c,id,food2h_calorie_mean,food2h_total_carb_sum,food2h_total_carb_mean,food2h_sugar_sum,food2h_sugar_mean,...,vfc_NNx,vfc_pNNx,temp_mean,temp_median,temp_max,temp_min,temp_std,temp_q1,temp_q3,ritcir_indicator_ritmo_circadiano
0,2020-02-13 17:20:00,PersNorm,FEMALE,5.5,1.0,0.0,0.0,0.0,0.0,0.0,...,92.0,65.7,33.604500,33.61,33.65,33.50,0.027977,33.590,33.63,0.0
1,2020-02-13 17:25:00,PersNorm,FEMALE,5.5,1.0,0.0,0.0,0.0,0.0,0.0,...,144.0,57.1,33.171867,33.21,33.57,32.73,0.266618,32.925,33.43,1.0
2,2020-02-13 17:30:00,PersNorm,FEMALE,5.5,1.0,0.0,0.0,0.0,0.0,0.0,...,112.0,60.9,33.136333,33.18,33.47,32.75,0.251755,32.870,33.39,1.0
3,2020-02-13 17:35:00,PersNorm,FEMALE,5.5,1.0,0.0,0.0,0.0,0.0,0.0,...,37.0,60.7,33.244767,33.25,33.39,33.11,0.052050,33.210,33.27,0.0
4,2020-02-13 17:40:00,PersNorm,FEMALE,5.5,1.0,0.0,0.0,0.0,0.0,0.0,...,65.0,52.4,33.315067,33.34,33.43,33.21,0.068227,33.250,33.37,0.0


In [6]:
label = 'nivel'
classes = dataset_df['nivel'].unique().tolist()
print(classes)

['PersNorm', 'PersHigh', 'PersLow']


In [7]:
# Split the dataset into a training and a testing dataset.

def split_dataset(dataset, test_ratio=0.30):
  """Splits a panda dataframe in two."""
  test_indices = np.random.rand(len(dataset)) < test_ratio
  return dataset[~test_indices], dataset[test_indices]

In [8]:
train_ds_pd, test_ds_pd = split_dataset(dataset_df)
print("{} examples in training, {} examples for testing.".format(
    len(train_ds_pd), len(test_ds_pd)))

24132 examples in training, 10460 examples for testing.


In [9]:
import ydf
model = ydf.GradientBoostedTreesLearner(label=label).train(train_ds_pd)

Train model on 24132 examples


max_vocab_count = -1 for column nivel, the dictionary will not be pruned by size.
"goss_alpha" set but "sampling_method" not equal to "GOSS".
"goss_beta" set but "sampling_method" not equal to "GOSS".
"selective_gradient_boosting_ratio" set but "sampling_method" not equal to "SELGB".
Data spec:
Number of records: 24132
Number of columns: 70

Number of columns by type:
	NUMERICAL: 67 (95.7143%)
	CATEGORICAL: 3 (4.28571%)

Columns:

NUMERICAL: 67 (95.7143%)
	3: "hba1c" NUMERICAL mean:5.75423 min:5.3 max:6.4 sd:0.27311 dtype:DTYPE_FLOAT64
	4: "id" NUMERICAL mean:7.77573 min:1 max:15 sd:4.20592 dtype:DTYPE_FLOAT64
	5: "food2h_calorie_mean" NUMERICAL mean:206.833 min:0 max:1360 sd:154.635 dtype:DTYPE_FLOAT64
	6: "food2h_total_carb_sum" NUMERICAL mean:18.4681 min:0 max:443.708 sd:30.1971 dtype:DTYPE_FLOAT64
	7: "food2h_total_carb_mean" NUMERICAL mean:26.6741 min:0 max:153.548 sd:21.4143 dtype:DTYPE_FLOAT64
	8: "food2h_sugar_sum" NUMERICAL mean:7.5518 min:0 max:144.038 sd:13.7225 dtype:DTYPE_

Model trained in 0:00:26.958263


	num-trees:300 train-loss:0.006068 train-accuracy:0.996136 valid-loss:0.009195 valid-accuracy:0.995816
Truncates the model to 891 tree(s) i.e. 297  iteration(s).
Final model num-trees:297 valid-loss:0.009161 valid-accuracy:0.995816


In [10]:
model.describe()

In [11]:
model.predict(test_ds_pd)

Model loaded with 891 root(s), 49239 node(s), and 69 input feature(s).
Engine "GradientBoostedTreesGeneric" built


array([[2.7215599e-05, 9.9997276e-01, 7.6759255e-10],
       [5.1939398e-05, 9.9994797e-01, 6.4103911e-10],
       [5.9104579e-05, 9.9994093e-01, 6.1931515e-10],
       ...,
       [4.4682303e-01, 5.5317682e-01, 2.5041030e-07],
       [4.4682303e-01, 5.5317682e-01, 2.5041030e-07],
       [4.4682303e-01, 5.5317682e-01, 2.5041030e-07]], dtype=float32)

In [12]:
# Analyse a model (e.g. partial dependence plot, variable importance)
model.analyze(test_ds_pd)

# Benchmark the inference speed of a model
model.benchmark(test_ds_pd)

Model loaded with 891 root(s), 49239 node(s), and 69 input feature(s).
Engine "GradientBoostedTreesGeneric" built
Run the model with the fast engine wrapper
List plotting attribute combinations
Found 69 combination(s)
Initiate PDP accumulator
Compute partial dependence plot for 69 set of features and 3080 model call(s) per example.
1 examples scanned.
Maximum duration reached. Interrupting analysis early.
List plotting attribute combinations
Found 69 combination(s)
Initiate CEP accumulator
Compute conditional expectation plot for 69 set of features and 3080 model call(s) per example.
1 examples scanned.
Model loaded with 891 root(s), 49239 node(s), and 69 input feature(s).
Engine "GradientBoostedTreesGeneric" built
Running 70 features on 12 threads with 1 rounds
Model loaded with 891 root(s), 49239 node(s), and 69 input feature(s).
Model loaded with 891 root(s), 49239 node(s), and 69 input feature(s).
Model loaded with 891 root(s), 49239 node(s), and 69 input feature(s).
Model loaded w

Single-thread inference time per example: 29.327 us (microseconds)
Details: 125520 predictions in 0.000 seconds

Multi-thread inference time per example: 3.859 us (microseconds)
Details: 721740 predictions in 0.000 seconds using 12 threads

* Measured with the C++ serving API. See model.to_cpp().

In [13]:
evaluation = model.evaluate(test_ds_pd)

# Query individual evaluation metrics
print(f"test accuracy: {evaluation.accuracy}")

# Show the full evaluation report
print("Full evaluation report:")
evaluation

test accuracy: 0.9949330783938815
Full evaluation report:


Label \ Pred,PersHigh,PersNorm,PersLow
PersHigh,5934,7,0
PersNorm,44,4239,1
PersLow,0,1,234
